# 1. Baseline

## Libraries

In [1]:
'''
!pip install torch
!pip install easydict
!pip install torchvision
!pip install opencv-python
'''

'\n!pip install torch\n!pip install easydict\n!pip install torchvision\n!pip install opencv-python\n'

In [2]:
import os
from pathlib import Path
from tqdm import tqdm # used for visualising the training part
from easydict import EasyDict as edict

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.utils as vutils
from torchvision import transforms as trans

from data.ms1m import get_train_loader
from data.lfw import LFW

from backbone.arcfacenet import SEResNet_IR
from margin.ArcMarginProduct import ArcMarginProduct

from util.utils import save_checkpoint, test

## Configuration

In [3]:
conf = edict()
conf.train_root = './dataset/MS1M'
conf.lfw_test_root = './dataset/lfw_aligned_112'
conf.lfw_file_list = './dataset/lfw_pair.txt'

conf.mode = 'se_ir' # 'ir'
conf.depth = 50   # 100 152
conf.margin_type = 'ArcFace'
conf.feature_dim = 512
conf.scale_size = 32.0
conf.batch_size = 96 # if the memory of your video card is less than 6 gb you can use it as 16 we use multiples of 16
conf.lr = 0.01 # there is a relation between learning rate and the batch size. linear but not exact
conf.milestones = [8, 10, 12]
conf.total_epoch = 14
conf.save_folder = './saved'
conf.save_dir = os.path.join(conf.save_folder, conf.mode + '_' + str(conf.depth)) # ./saved/se_ir_50
conf.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
conf.num_workers = 4
conf.pin_memory =  True

In [4]:
os.makedirs(conf.save_dir, exist_ok = True)

## Data Loader

In [5]:
transform = trans.Compose([
    trans.ToTensor(),  # range [0,255] -> [0.0, 1.0]
    trans.Normalize(mean = (0.5,0.5,0.5), std = (0.5,0.5,0.5))
])

trainloader, class_num = get_train_loader(conf)                   

In [6]:
print('number of id: ', class_num)

number of id:  1


In [7]:
print(trainloader.dataset)

Dataset ImageFolder
    Number of datapoints: 29148
    Root location: ./dataset/MS1M
    StandardTransform
Transform: Compose(
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
           )


In [8]:
lfwdataset = LFW(conf.lfw_test_root, conf.lfw_file_list, transform = transform)
lfwloader = torch.utils.data.DataLoader(lfwdataset, batch_size = 64, num_workers = conf.num_workers)                                

# Model

In [9]:
print(conf.device)

cpu


In [10]:
print(torch.version.cuda)

None


In [11]:
net = SEResNet_IR(conf.depth, feature_dim = conf.feature_dim, mode = conf.mode).to(conf.device)
margin = ArcMarginProduct(conf.feature_dim, class_num).to(conf.device)

In [12]:
print(net)

SEResNet_IR(
  (input_layer): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): PReLU(num_parameters=64)
  )
  (output_layer): Sequential(
    (0): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Dropout(p=0.4, inplace=False)
    (2): Flatten()
    (3): Linear(in_features=25088, out_features=512, bias=True)
    (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (body): Sequential(
    (0): BottleNeck_IR_SE(
      (shortcut_layer): MaxPool2d(kernel_size=1, stride=2, padding=0, dilation=1, ceil_mode=False)
      (res_layer): Sequential(
        (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (2): BatchNorm2d(64, eps=1e-05, moment

In [13]:
criterion = nn.CrossEntropyLoss()

In [14]:
optimizer = optim.SGD([
    {'params': net.parameters(), 'weight_decay': 5e-4},
    {'params': margin.parameters(), 'weight_decay': 5e-4}
], lr = conf.lr, momentum = 0.9, nesterov = True)

In [15]:
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.01
    maximize: False
    momentum: 0.9
    nesterov: True
    weight_decay: 0.0005

Parameter Group 1
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.01
    maximize: False
    momentum: 0.9
    nesterov: True
    weight_decay: 0.0005
)


In [16]:
def schedule_lr():
    for params in optimizer.param_groups:
        params['lr'] /= 10
    print(optimizer, flush = True)    

## Training

In [17]:
best_acc = 0
for epoch in range(1, conf.total_epoch + 1):
    net.train()
    print('epoch {}/{}'.format(epoch, conf.total_epoch), flush = True)
    if epoch == conf.milestones[0]: #8
        schedule_lr()
    if epoch == conf.milestones[1]: #10
        schedule_lr()    
    if epoch == conf.milestones[2]: #12
        schedule_lr()    

    for data in tqdm(trainloader):
        img, label = data[0].to(conf.device), data[1].to(conf.device)
        optimizer.zero_grad()

        logits = net(img)
        output = margin(logits, label)
        total_loss = criterion(output, label)
        total_loss.backward()
        optimizer.step()

        #test
        
        net.eval()

        lfw_acc = test(conf, net, lfwloader)
        print('/nLFW {:.4f} | train_loss: {:.4f}/n'.format(lfw_acc, total_loss.item()))
        
        is_best = lfw_acc > best_acc
        best_acc = max( lfw_acc, best_acc)
        
        save_chechpoint({
            'epoch': epoch,
            'net_state_dict': net.state_dict(),
            'best_acc': best_acc       
        }, is_best, checkpooint = conf.save_dir)

epoch 1/14


C:\Users\Hasan\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
  0%|          | 0/304 [00:30<?, ?it/s]


TypeError: test() missing 1 required positional argument: 'lfwloader'